In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import joblib
import os
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.utils import compute_class_weight

In [7]:
df = pd.read_csv("rain.csv", parse_dates = ["Date"])
df = df.sort_values(["Location", "Date"]).reset_index(drop = True)

df["RainToday"] = df["RainToday"].map({"Yes" : 1, "No" : 0})
df["RainTomorrow"] = df["RainTomorrow"].map({"Yes" : 1, "No" : 0})

df = df.groupby("Location").apply(lambda g : g.ffill().bfill()).reset_index(drop = True)

numeric_cols = [
    "MinTemp","MaxTemp","Humidity9am","Humidity3pm",
    "Pressure9am","Pressure3pm","WindSpeed9am","WindSpeed3pm","RainToday"
]

C:\Users\kanan\AppData\Local\Temp\ipykernel_19924\3457086812.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("Location").apply(lambda g : g.ffill().bfill()).reset_index(drop = True)


In [8]:
def create_seq(data, lookback, numeric_cols, target_col) :
    x, y = [], []

    for i in range (lookback, len(data)):
        seq = df[numeric_cols].iloc[i - lookback : i].values
        x.append(seq)
        y.append(df[target_col].iloc[i])

    return np.array(x), np.array(y)

In [13]:
lookback = 15
results = {}

for loc, grp in df.groupby("Location"):
    print(f"--Training for {loc}--")
    g = grp.reset_index(drop=True)
    if len(g) < lookback + 10:
        continue

    scaler = StandardScaler()
    g_scaled = g.copy()
    g_scaled[numeric_cols] = scaler.fit_transform(g[numeric_cols])

    x, y = create_seq(g_scaled, lookback, numeric_cols, "RainTomorrow")

    split = int(0.8 * len(x))
    x_train, x_test = x[:split], x[split:]
    y_train, y_test = y[:split], y[split:]

    classes = np.unique(y_train)
    cw_vals = compute_class_weight("balanced", classes = classes, y=y_train)
    class_weights = {int(classes[i]) : float(cw_vals[i]) for i in range(len(classes))}

    model = Sequential([
        Masking(mask_value = 0, input_shape = (lookback, len(numeric_cols))),
        LSTM(64, return_sequences = False),
        Dropout(0.25),
        Dense(32, activation = 'relu'),
        Dense(1, activation = 'sigmoid')
    ])
    model.compile(optimizer = Adam(0.001), loss = "binary_crossentropy", metrics = ["accuracy"])

    es = EarlyStopping(monitor = "val_loss", patience = 3, restore_best_weights=True)

    history = model.fit(x_train, y_train, epochs = 20, batch_size = 32, class_weight = class_weights, callbacks = [es], verbose = 1)

    y_pred_prob = model.predict(x_test).ravel()
    y_pred = (y_pred_prob > 5).astype(int)

    acc = accuracy_score(y_test, y_pred)
    print(f"\naccuracy : {acc}")
    print(classification_report(y_test, y_pred))

--Training for Bangalore--

accuracy : 0.6585365853658537
              precision    recall  f1-score   support

           0       0.70      0.90      0.79        29
           1       0.25      0.08      0.12        12

    accuracy                           0.66        41
   macro avg       0.48      0.49      0.46        41
weighted avg       0.57      0.66      0.59        41

--Training for Chennai--

accuracy : 0.7105263157894737
              precision    recall  f1-score   support

           0       0.71      1.00      0.83        27
           1       0.00      0.00      0.00        11

    accuracy                           0.71        38
   macro avg       0.36      0.50      0.42        38
weighted avg       0.50      0.71      0.59        38

--Training for Kolkata--


D:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



accuracy : 0.6111111111111112
              precision    recall  f1-score   support

           0       0.69      0.85      0.76        26
           1       0.00      0.00      0.00        10

    accuracy                           0.61        36
   macro avg       0.34      0.42      0.38        36
weighted avg       0.50      0.61      0.55        36

--Training for Mumbai--

accuracy : 0.6666666666666666
              precision    recall  f1-score   support

           0       0.69      0.96      0.80        25
           1       0.00      0.00      0.00        11

    accuracy                           0.67        36
   macro avg       0.34      0.48      0.40        36
weighted avg       0.48      0.67      0.56        36

--Training for New Delhi--

accuracy : 0.7027027027027027
              precision    recall  f1-score   support

           0       0.74      0.93      0.83        28
           1       0.00      0.00      0.00         9

    accuracy                          

In [22]:
# ============================================
# 1. IMPORTS
# ============================================
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE

# ============================================
# 2. LOAD DATA
# ============================================
df = pd.read_csv("rain.csv")   # or your rain dataset

print(df.shape)
df.head()

# ============================================
# 3. CLEAN DATA
# ============================================

# Drop columns with too many missing values
df = df.drop(columns=["Evaporation","Sunshine","Cloud3pm","Cloud9am"], errors="ignore")

# Drop rows missing the target
df = df.dropna(subset=["RainTomorrow"])

# Convert target to binary
df["RainTomorrow"] = df["RainTomorrow"].map({"Yes":1,"No":0})

# Fill numeric missing values with mean
for col in df.select_dtypes(include=np.number):
    df[col] = df[col].fillna(df[col].mean())

# Fill categorical missing with mode
for col in df.select_dtypes(include="object"):
    df[col] = df[col].fillna(df[col].mode()[0])

print("Data cleaned:", df.shape)

# ============================================
# 4. ONE-HOT ENCODE CATEGORICAL
# ============================================
df = pd.get_dummies(df, drop_first=True)

print("After encoding:", df.shape)

# ============================================
# 5. SPLIT DATA
# ============================================
X = df.drop("RainTomorrow", axis=1)
y = df["RainTomorrow"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train target distribution:")
print(y_train.value_counts())

# ============================================
# 6. FIX IMBALANCE WITH SMOTE
# ============================================
sm = SMOTE(random_state=42)
X_train_bal, y_train_bal = sm.fit_resample(X_train, y_train)

print("After SMOTE:")
print(y_train_bal.value_counts())

# ============================================
# 7. SCALE FEATURES
# ============================================
scaler = StandardScaler()
X_train_bal = scaler.fit_transform(X_train_bal)
X_test = scaler.transform(X_test)

# ============================================
# 8. TRAIN XGBOOST MODEL
# ============================================
model = XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric="logloss"
)

model.fit(X_train_bal, y_train_bal)

# ============================================
# 9. EVALUATE
# ============================================
y_pred = model.predict(X_test)

print("\n=============== CLASSIFICATION REPORT ===============\n")
print(classification_report(y_test, y_pred))

print("\n=============== CONFUSION MATRIX ===============\n")
print(confusion_matrix(y_test, y_pred))


(1000, 12)
Data cleaned: (1000, 12)
After encoding: (1000, 1013)
Train target distribution:
RainTomorrow
0    580
1    220
Name: count, dtype: int64
After SMOTE:
RainTomorrow
0    580
1    580
Name: count, dtype: int64

=============== CLASSIFICATION REPORT ===============

              precision    recall  f1-score   support

           0       0.93      0.87      0.90       145
           1       0.71      0.84      0.77        55

    accuracy                           0.86       200
   macro avg       0.82      0.85      0.83       200
weighted avg       0.87      0.86      0.86       200


=============== CONFUSION MATRIX ===============

[[126  19]
 [  9  46]]


In [4]:
# ============================================
# 1. IMPORTS
# ============================================
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, TimeSeriesSplit

from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE

# ============================================
# 2. LOAD DATA
# ============================================
df = pd.read_csv("rain.csv", parse_dates=["Date"])
df = df.sort_values(["Location", "Date"]).reset_index(drop=True)

# Convert target
df["RainTomorrow"] = df["RainTomorrow"].map({"Yes": 1, "No": 0})

# ============================================
# 3. CLEAN DATA
# ============================================
numeric_cols = df.select_dtypes(include=np.number).columns.tolist()

for col in numeric_cols:
    df[col] = df[col].fillna(df[col].mean())

for col in df.select_dtypes(include="object"):
    df[col] = df[col].fillna(df[col].mode()[0])

# ============================================
# 4. FUNCTION TO ADD TIME-SERIES FEATURES
# ============================================

def add_time_features(g):
    g = g.sort_values("Date").reset_index(drop=True)

    # Lag features
    for lag in [1, 2, 3, 7, 14]:
        for col in numeric_cols:
            g[f"{col}_lag{lag}"] = g[col].shift(lag)

    # Rolling means
    for win in [3, 7, 14]:
        for col in numeric_cols:
            g[f"{col}_roll{win}"] = g[col].rolling(win).mean()

    # Rolling std
    for win in [7, 14]:
        for col in numeric_cols:
            g[f"{col}_std{win}"] = g[col].rolling(win).std()

    # Rolling min/max
    for win in [7, 14]:
        for col in numeric_cols:
            g[f"{col}_min{win}"] = g[col].rolling(win).min()
            g[f"{col}_max{win}"] = g[col].rolling(win).max()

    return g

# ============================================
# 5. APPLY FEATURES PER LOCATION
# ============================================

df_ts = df.groupby("Location").apply(add_time_features).reset_index(drop=True)
df_ts = df_ts.dropna().reset_index(drop=True)

print("New shape with time-series features:", df_ts.shape)

# ============================================
# 6. ONE-HOT ENCODE CATEGORICAL
# ============================================
df_ts = pd.get_dummies(df_ts, drop_first=True)

# ============================================
# 7. TRAIN/TEST SPLIT (TIME-AWARE)
# ============================================

train_size = int(0.8 * len(df_ts))

train = df_ts.iloc[:train_size]
test = df_ts.iloc[train_size:]

X_train = train.drop("RainTomorrow", axis=1)
y_train = train["RainTomorrow"]

X_test = test.drop("RainTomorrow", axis=1)
y_test = test["RainTomorrow"]

# ============================================
# 8. BALANCE WITH SMOTE
# ============================================
sm = SMOTE()
X_train_np = X_train.to_numpy()
y_train_np = y_train.to_numpy()
X_train_bal, y_train_bal = sm.fit_resample(X_train_np, y_train_np)


# ============================================
# 9. SCALE FEATURES
# ============================================
scaler = StandardScaler()
X_train_bal = scaler.fit_transform(X_train_bal)
# Convert test data to numpy array as well
X_test_np = X_test.to_numpy()
X_test = scaler.transform(X_test_np)

# ============================================
# 10. TRAIN XGBOOST WITH TIME-SERIES FEATURES
# ============================================
model = XGBClassifier(
    n_estimators=350,
    max_depth=8,
    learning_rate=0.03,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric="logloss",
    # Add this parameter to handle the data type issue
    enable_categorical=True
)


model.fit(X_train_bal, y_train_bal)

# ============================================
# 11. EVALUATE
# ============================================

y_pred = model.predict(X_test)

print("\n========== CLASSIFICATION REPORT =============\n")
print(classification_report(y_test, y_pred))

print("\n========== CONFUSION MATRIX =============\n")
print(confusion_matrix(y_test, y_pred))


C:\Users\kanan\AppData\Local\Temp\ipykernel_19924\1187580270.py:59: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  g[f"{col}_min{win}"] = g[col].rolling(win).min()
C:\Users\kanan\AppData\Local\Temp\ipykernel_19924\1187580270.py:60: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  g[f"{col}_max{win}"] = g[col].rolling(win).max()
C:\Users\kanan\AppData\Local\Temp\ipykernel_19924\1187580270.py:59: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perform

New shape with time-series features: (930, 138)


TypeError: float() argument must be a string or a real number, not 'Timestamp'

In [21]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight

# --------------------------
# 1. Load Data
# --------------------------
df = pd.read_csv("rain.csv")

numeric_cols = [
    "MinTemp","MaxTemp","Humidity9am","Humidity3pm",
    "Pressure9am","Pressure3pm","WindSpeed9am","WindSpeed3pm","RainToday"
]
target_col = "RainTomorrow"

df = df[numeric_cols + [target_col]].dropna().reset_index(drop=True)

# --------------------------
# 2. Encode Target
# --------------------------


df["RainToday"] = df["RainToday"].map({"No": 0, "Yes": 1})
df["RainTomorrow"] = df["RainTomorrow"].map({"No": 0, "Yes": 1})


# --------------------------
# 3. Scale Only Inputs
# --------------------------
scaler = StandardScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

# --------------------------
# 4. Create Sequences
# --------------------------
def create_sequences(data, lookback, feature_cols, target_col):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[feature_cols].iloc[i:i+lookback].values)
        y.append(data[target_col].iloc[i+lookback])
    return np.array(X), np.array(y)

lookback = 14  # last 14 days to predict tomorrow
X, y = create_sequences(df, lookback, numeric_cols, target_col)

print("X shape:", X.shape)   # (samples, 14, 9)
print("y shape:", y.shape)   # (samples,)

# --------------------------
# 5. Train/Test Split
# --------------------------
split = int(0.8 * len(X))
x_train, x_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# --------------------------
# 6. Handle Class Imbalance
# --------------------------
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights = dict(enumerate(class_weights))
print("Class Weights:", class_weights)

# --------------------------
# 7. Build LSTM Model
# --------------------------
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(lookback, len(numeric_cols))),
    Dropout(0.3),
    LSTM(32),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=Adam(0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

# --------------------------
# 8. Train Model
# --------------------------
history = model.fit(
    x_train, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    class_weight=class_weights,
    verbose=1
)

# --------------------------
# 9. Evaluate
# --------------------------
y_pred_prob = model.predict(x_test).ravel()
y_pred = (y_pred_prob > 0.5).astype(int)

print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))


X shape: (986, 14, 9)
y shape: (986,)
Class Weights: {0: np.float64(0.696113074204947), 1: np.float64(1.7747747747747749)}


D:\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 14, 64)         │        18,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 14, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,905 (124.63 KB)

 Trainable params: 31,905 (124.63 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - accuracy: 0.6095 - loss: 0.6812 - val_accuracy: 0.4684 - val_loss: 0.6952
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6762 - loss: 0.6725 - val_accuracy: 0.4494 - val_loss: 0.6931
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7016 - loss: 0.6696 - val_accuracy: 0.4747 - val_loss: 0.6944
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7032 - loss: 0.6664 - val_accuracy: 0.4747 - val_loss: 0.7021
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6175 - loss: 0.6622 - val_accuracy: 0.4747 - val_loss: 0.7059
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6587 - loss: 0.6573 - val_accuracy: 0.4873 - val_loss: 0.7138
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6206 - loss: 0.6531 - val_accuracy: 0.4557 - val_loss: 0.7275
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6190 - loss: 0.6504 - val_accuracy: 0.4557 - v

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.utils.class_weight import compute_class_weight

import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
df = pd.read_csv("rain.csv")  
df.head()

,Date,Location,MinTemp,MaxTemp,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,WindSpeed9am,WindSpeed3pm,RainToday,RainTomorrow
0,2025-01-01,Mumbai,20.6,28.7,67,37,1001.4,1007.0,10,13,No,No
1,2025-01-02,New Delhi,29.3,35.8,46,34,1008.3,1005.7,10,5,No,No
2,2025-01-03,Mumbai,26.0,42.5,74,89,1009.6,998.9,16,23,Yes,Yes
3,2025-01-04,Kolkata,24.0,39.6,94,38,1010.6,1002.3,16,9,No,No
4,2025-01-05,New Delhi,17.3,41.1,67,30,1005.6,999.4,14,23,Yes,No


In [4]:
df = df.ffill().bfill()

In [7]:
label_cols = ["Location", "RainToday", "RainTomorrow"]

le_dict = {}   # store encoders for each column

for col in label_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    le_dict[col] = le


In [8]:
features = [
    "MinTemp", "MaxTemp", "Humidity9am", "Humidity3pm",
    "Pressure9am", "Pressure3pm",
    "WindSpeed9am", "WindSpeed3pm",
    "RainToday", "Location"
]

target = "RainTomorrow"

X_raw = df[features]
y = df[target].values


In [9]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_raw)

X = pd.DataFrame(X_scaled, columns=features)
X.head()

,MinTemp,MaxTemp,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,WindSpeed9am,WindSpeed3pm,RainToday,Location
0,0.369128,0.180905,0.457627,0.118644,0.093333,0.600000,0.263158,0.421053,0.0,0.75
1,0.953020,0.537688,0.101695,0.067797,0.553333,0.513333,0.263158,0.000000,0.0,1.00
2,0.731544,0.874372,0.576271,1.000000,0.640000,0.060000,0.578947,0.947368,1.0,0.75
3,0.597315,0.728643,0.915254,0.135593,0.706667,0.286667,0.578947,0.210526,0.0,0.50
4,0.147651,0.804020,0.457627,0.000000,0.373333,0.093333,0.473684,0.947368,1.0,1.00


In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

# LSTM requires 3D input → (samples, time_steps, features)
X_train = np.reshape(X_train.values, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test.values, (X_test.shape[0], 1, X_test.shape[1]))

In [12]:
weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y),
    y=y
)

class_weights = {i: weights[i] for i in range(len(np.unique(y)))}
class_weights

{0: np.float64(0.6896551724137931), 1: np.float64(1.8181818181818181)}

In [13]:
model = Sequential([
    Input(shape=(1, len(features))),
    LSTM(64, return_sequences=False),
    Dropout(0.3),

    Dense(32, activation='relu'),
    Dropout(0.2),

    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 64)             │        19,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,313 (83.25 KB)

 Trainable params: 21,313 (83.25 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
es = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    X_train, y_train,
    epochs=80,
    batch_size=16,
    validation_split=0.2,
    callbacks=[es],
    class_weight=class_weights,
    verbose=1
)

Epoch 1/80
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.6047 - loss: 0.6773 - val_accuracy: 0.7250 - val_loss: 0.6772
Epoch 2/80
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7188 - loss: 0.6575 - val_accuracy: 0.8000 - val_loss: 0.6446
Epoch 3/80
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6906 - loss: 0.6214 - val_accuracy: 0.7125 - val_loss: 0.6118
Epoch 4/80
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7219 - loss: 0.5776 - val_accuracy: 0.7688 - val_loss: 0.5402
Epoch 5/80
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7812 - loss: 0.4969 - val_accuracy: 0.8062 - val_loss: 0.4561
Epoch 6/80
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7828 - loss: 0.4418 - val_accuracy: 0.8562 - val_loss: 0.3974
Epoch 7/80
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8391 - loss: 0.3873 - val_accuracy: 0.8625 - val_loss: 0.3649
Epoch 8/80
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8313 - loss: 0.3543 - val_accuracy: 0.8813 - val_loss

In [26]:
score = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8800 - loss: 0.2456 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


In [28]:
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92       148
           1       0.75      0.81      0.78        52

    accuracy                           0.88       200
   macro avg       0.84      0.86      0.85       200
weighted avg       0.88      0.88      0.88       200

